In [14]:
import pandas as pd
from sqlalchemy import create_engine

In [17]:
elon_file = "Resources/elonmusk_tweets.csv"
spacex_file = "Resources/spacex_launch_data.csv"
teslatwo_file="TSLA _to_29th.csv"

In [19]:
elon_df = pd.read_csv(elon_file)
spacex_df = pd.read_csv(spacex_file)
teslatwo_df=pd.read_csv(teslatwo_file)

In [20]:
spacex_df.head( )

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute)
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NaN,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525,LEO (ISS),NASA (COTS),Success,No attempt
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [21]:
spacex_cols = ["Flight Number", "Date","Launch Site", "Payload", "Customer", "Mission Outcome", "Landing Outcome"]
spacex_transform = spacex_df[spacex_cols].copy()

spacex_transform = spacex_transform.rename(columns={"Flight Number":"id",
                                                    "Date":"real_date",
                                                    "Launch Site": "launch_site",
                                                    "Payload":"payload",
                                                    "Customer":"customer",
                                                    "Mission Outcome":"mission_outcome",
                                                    "Landing Outcome":"landing_outcome"})

spacex_transform.drop_duplicates("id", inplace=True)
spacex_transform.set_index("id", inplace=True)

spacex_transform.head()


,real_date,launch_site,payload,customer,mission_outcome,landing_outcome
id,,,,,,
1,2010-06-04,CCAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success,Failure (parachute)
2,2010-12-08,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NASA (COTS) NRO,Success,Failure (parachute)
3,2012-05-22,CCAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success,No attempt
4,2012-10-08,CCAFS LC-40,SpaceX CRS-1,NASA (CRS),Success,No attempt
5,2013-03-01,CCAFS LC-40,SpaceX CRS-2,NASA (CRS),Success,No attempt


In [22]:
teslatwo_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [23]:
teslatwo_cols = ["Date","Open","High","Low","Close","Adj Close","Volume"]
tesla_transform = teslatwo_df[teslatwo_cols].copy()

tesla_transform = tesla_transform.rename(columns={"Date":"real_date",
                                                  "Open":"open_price",
                                                  "High":"high_price",
                                                  "Low":"low_price",
                                                  "Close":"closing_price",
                                                  "Adj Close":"adj_close",
                                                  "Volume":"volume"})

tesla_transform.drop_duplicates("real_date", inplace=True)
tesla_transform.set_index("real_date")
tesla_transform.head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [24]:
#tesla_transform.sort_values(by='real_date', ascending=False)

In [25]:
tesla_transform["range"] = tesla_transform["high_price"].subtract(tesla_transform["low_price"], fill_value=0)

In [26]:
tesla_transform.head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume,range
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,7.459999
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,7.120001
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,5.650000
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,4.390001
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900,4.170000


In [27]:
min_range = tesla_transform["range"].min()
max_range = tesla_transform["range"].max()
avg_range = tesla_transform["range"].mean()

In [28]:
tesla_transform["rng_perct_close"] =(tesla_transform["range"]/tesla_transform["closing_price"])*100

In [30]:
tesla_transform.sort_values(by='real_date',ascending=True).head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume,range,rng_perct_close
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,7.459999,31.226452
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,7.120001,29.878309
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,5.650000,25.728599
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,4.390001,22.864587
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900,4.170000,25.884542


In [32]:
tesla_transform.to_csv("teslanew.csv")

In [28]:
elon_df.head()

,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [29]:
elon_cols = ["id", "created_at", "text"]
elon_transform = elon_df[elon_cols].copy()

elon_transform = elon_transform.rename(columns={"id":"id","created_at":"real_date",
                                         "text":"tweet_text"})
elon_transform.drop_duplicates("id",inplace=True)
elon_transform.head()





,id,real_date,tweet_text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [30]:
elon_transform["real_date"]=elon_transform.real_date.str.split(expand=True)

In [31]:
elon_transform.head()

,id,real_date,tweet_text
0,849636868052275200,2017-04-05,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [32]:
elon_transform.dropna(how="any")

,id,real_date,tweet_text
0,849636868052275200,2017-04-05,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."
5,848415731502923777,2017-04-02,"b""@Lexxxzis It's just a helicopter in helicopt..."
6,848415356263702528,2017-04-02,"b""@verge It won't matter"""
7,848398971139629057,2017-04-02,b'@SuperCoolCube Pretty good'
8,848244577521647616,2017-04-01,"b""Why did we waste so much time developing sil..."
9,848243350993895424,2017-04-01,b'Technology breakthrough: turns out chemtrail...


In [73]:
spacex_transform.head()

,real_date,launch_site,payload,customer,mission_outcome,landing_outcome
id,,,,,,
1,2010-06-04,CCAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success,Failure (parachute)
2,2010-12-08,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NASA (COTS) NRO,Success,Failure (parachute)
3,2012-05-22,CCAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success,No attempt
4,2012-10-08,CCAFS LC-40,SpaceX CRS-1,NASA (CRS),Success,No attempt
5,2013-03-01,CCAFS LC-40,SpaceX CRS-2,NASA (CRS),Success,No attempt


In [33]:
spacex_transform.to_csv("spacexnew.csv")

In [14]:
tesla_transform.dropna(how="any")

,real_date,closing_price,volume
0,2010-06-29,23.889999,18766300
1,2010-06-30,23.830000,17187100
2,2010-07-01,21.959999,8218800
3,2010-07-02,19.200001,5139800
4,2010-07-06,16.110001,6866900
5,2010-07-07,15.800000,6921700
6,2010-07-08,17.459999,7711400
7,2010-07-09,17.400000,4050600
8,2010-07-12,17.049999,2202500
9,2010-07-13,18.139999,2680100
